In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

## Import framework main libs

In [ ]:
from tmqrfeed.manager import DataManager
from tmqrstrategy.optimizers import OptimizerBase, OptimizerGenetic


## Import strategy from module

In [ ]:
from tmqralphas.strategy_dsp_lpbp_combination import Strategy_DSP_LPBP_Combination

# Strategy inheritance trick!

We inherited Strategy_DSP_LPBP_Combination strategy the logic remains the same, but position management in different.

In [ ]:
import pandas as pd
from tmqrfeed.quotes import QuoteIndex
from tmqr.logs import log
from tmqr.errors import PositionNotFoundError


class Strategy_DSP_LPBP_HedgedByIndex(Strategy_DSP_LPBP_Combination):
    def setup(self):
        # Call parent Strategy_DSP_LPBP_Combination.setup() -> tmqrstrategy.strategy_alpha.StrategyAlpha.setup()
        super().setup()
        
        #
        # We have to add index we wanted to hedge by
        #        
        
        HEDGE_IDX_NAME = self.context['index_hedge_name']        
        self.dm.series_extra_set('index_hedge', QuoteIndex, HEDGE_IDX_NAME, set_session=True, check_session=True)
        
        #
        # Pre-calculate hedge ratios
        #
        try:
            primary_quotes = self.dm.quotes()['c']

            # Adjusting primary quotes to dollars for raw future prices (in case of Continuous future EXO)
            instrument_name = self.context['index_name'].split('_')[0]
            
            instrument_info = self.dm.instrument_info_get(instrument_name)
            instrument_point_value = 1.0 / instrument_info.ticksize * instrument_info.tickvalue
            
            primary_quotes *= instrument_point_value
        except KeyError:
            # Use RAW EXO prices because they are already in dollars!
            primary_quotes = self.dm.quotes()['equity_decision']
            
        try:
            secondary_quotes = self.dm.quotes('index_hedge')['c']
            
            # Adjusting hedge index quotes to dollars for raw future prices (in case of Continuous future EXO)
            instrument_name = HEDGE_IDX_NAME.split('_')[0]
            
            instrument_info = self.dm.instrument_info_get(instrument_name)
            instrument_point_value = 1.0 / instrument_info.ticksize * instrument_info.tickvalue
            
            secondary_quotes *= instrument_point_value
        except KeyError:
            # Use RAW EXO prices because they are already in dollars!
            secondary_quotes = self.dm.quotes('index_hedge')['equity_decision']
        
        hedge_type = self.context.get('hedge_type', '')
        hedge_window = self.context.get('hedge_window', None)

        if hedge_type == 'vola':
            # Volatility based hedging
            primary_usd_vola = (primary_quotes.diff()).rolling(hedge_window).std()
            secondary_usd_vola = (secondary_quotes.diff()).rolling(hedge_window).std()

            # Calculating USD value ratio per 10 contracts        
            self.hedge_ratio = primary_usd_vola / secondary_usd_vola
        elif hedge_type == 'beta':
            # Beta based hedging
            rets_primary = primary_quotes.diff()
            rets_secondary = secondary_quotes.diff()

            sigma_primary = rets_primary.rolling(hedge_window).std()
            sigma_secondary = rets_secondary.rolling(hedge_window).std()

            cor = rets_primary.rolling(hedge_window).corr(rets_secondary)

            # Calculating hedge ratio, i.e. BETA = Correlation(A, B) * (StDev(A) / StDev(B))
            self.hedge_ratio = cor * (sigma_primary / sigma_secondary)
        else:
            raise SettingsError(f"Unexpected hedge type: '{hedge_type}', only 'vola' and 'beta' allowed")
    
    #
    # This is exact copy/paste of souce code of tmqrstrategy.strategy_alpha.StrategyAlpha.calculate_position() method
    #
    def calculate_position(self, date: datetime, exposure_record: pd.DataFrame):
        """
        This alpha just replicates EXO/SmartEXO index position


        This method used for position construction based on exposure information returned from calculate(),
        here you can initiate (replicate) EXO index position or setup any custom position you want.
        """
        # Get the position of Quote algo (in this case current cont futures)
        primary_quotes_position = self.dm.position()

        # ALSO you can get secondary positions
        # secondary_position = self.dm.position('CONTFUT')

        # get net exposure for all members
        # exposure_record - is a slice of exposures results of picked alphas at 'date'

        # We are calling sum() because we have multiple records of 'exposure'
        # 1-alpha member of best in the swarm per row
        if 'exposure' not in exposure_record:
            raise StrategyError(
                "'exposure_record' expected to have 'exposure' column, check alpha's calculate(...) method "
                "to make sure that it returns valid pandas.DataFrame with exposure column or just check "
                "for 'return self.exposure(...)' in the last line")
        exposure = exposure_record['exposure'].sum()

        #
        # Just replicate primary quotes position
        #
        replicated_pos = primary_quotes_position.get_net_position(date)
        self.position.add_net_position(date, replicated_pos, qty=exposure)
        
        
        #
        # Do new position management magic here
        #
        index_hedge_position = self.dm.position('index_hedge')
        try:
            hedge_position_rec = index_hedge_position.get_net_position(date)
            
            hedge_ratio = self.hedge_ratio.loc[date]
            
            if math.isnan(hedge_ratio) or hedge_ratio == 0:
                # Skipping 'NaN' hedge ratios (usualy days before full hedge_window)
                log.debug(f"{date}: Skipping 'NaN' or zero hedge ratios")
                return
            
            hedge_size = round(abs(exposure)*hedge_ratio*self.context['index_hedge_direction'])
            
            # Add index position as hedge
            # NOTE: exposure - is a alpha's exporure of trade, when alpha is out of market
            #                  exposure equals 0, this means that means no position and hedge
            # NOTE: self.context['index_hedge_direction'] allowed 1, -1, or even 0 - i.e. no hedge
            self.position.add_net_position(date, hedge_position_rec, qty=hedge_size)
        except PositionNotFoundError as exc:
            log.error(f"Couldn't find hedged index position! {exc}")
            
        #print(f'Exposure: {exposure}')
        #print(self.position)
        
    

In [ ]:
ALPHA_CONTEXT = {
        'name': 'HO_NewStrategy_DSP_LPBP_Combination_With_IndexHedge', # Global alpha name, which be used for load/save from DB
        'context': { # Strategy specific settings
            # These settings only applycable for alphas derived from StrategyAlpha strategy 
            # StrategyAlpha - is a classic EXO/SmartEXO based alpha
            'index_name': 'US.HO-US.CL_EXOSpreadHedged',      # Name of EXO index to trade
            
            # !!! NEW RECORD
            'index_hedge_name': 'US.CL_EXOSemiFuture_Delta25', # Name of the index used for hedge
            'index_hedge_direction': 0, # ALLOWED 1, -1, or even 0 - i.e. no hedge
            'hedge_type': 'beta', # 'vola' - stdev(returns) volatility hedge, 'beta' - Beta coef. hedge,
            'hedge_window': 200,  # number of periods to calculate hedge coef. 
            #
            
            'costs_per_option': 3.0,
            'costs_per_contract': 3.0,
        },
        'wfo_params': {
            'window_type': 'rolling',  # Rolling window for IIS values: rolling or expanding
            'period': 'M',  # Period of rolling window 'M' - monthly or 'W' - weekly
            'oos_periods': 12,  # Number of months is OOS period
            'iis_periods': 4,
            # Number of months in IIS rolling window (only applicable for 'window_type' == 'rolling')
        },
        'wfo_optimizer_class': OptimizerGenetic, 
        'wfo_optimizer_class_kwargs': {
            'nbest_count': 1,
            'nbest_fitness_method': 'max',
            'population_size': 15, 
            'number_generations': 2, 
            'rand_seed': 1, # Uncomment this parameter to make genetic results repeatable
        },
        'wfo_opt_params': [
                ('Direction', [1]),
                ('LP order',[2]),
                ('LP freq',[0.5982, 0.5924, 0.5937, 0.5938, 0.5908]), # 0 > f < 1
                ('BP order',[12]),
                ('BP start freq', [0.1986]),
                ('BP stop freq',   [0.2]),
                ('BP multiplier',[-4, -3] ),
                ('Rule index',   [3] ),                        
            ],
        'wfo_members_count': 1,
        'wfo_costs_per_contract': 0.0,
        'wfo_scoring_type': 'netprofit'
    }

# Run the alpha

In [ ]:
# DataManager is a core class of the framework
dm = DataManager()

# Init alpha class and run
alpha = Strategy_DSP_LPBP_HedgedByIndex(dm, **ALPHA_CONTEXT)    

In [ ]:
alpha.run()

# Equity

In [ ]:
alpha.stats['series']['equity'].plot()

# Hedging information


### Hedge ratios

In [ ]:
alpha.hedge_ratio.plot()

### Index hedge price

In [ ]:
alpha.dm.quotes('index_hedge')['equity_decision'].plot()

# Position holdings

#### Position hedge proof: alpha uses ES furues as primary position and options (from index position) as hedge

In [ ]:
for dt, pos_rec in alpha.position._position.items():
    print(f'\n{dt}')
    for asset, rec in pos_rec.items():
        print('\t{0:<50} {1:>5g}'.format(str(asset), rec[2]))